In [1]:
# https://www.kaggle.com/competitions/playground-series-s3e23/overview

import sys
# sys.path.append("../utils")
from utils import Util
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
sample = pd.read_csv("./data/sample_submission.csv")

c:\Users\nakamura\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# 1回目 
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 特徴量と目的変数の分割
X = train.drop('defects', axis=1)
y = train['defects']

# データの分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# モデルの学習
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X_train, y_train)

# 評価用データの予測
X_test = test
y_pred = model.predict(X_test)

# 予測結果の表示
print(y_pred)

# 結果をCSVファイルとして保存
result = pd.DataFrame(y_pred).astype(int)
result.columns = ['defects']
submit = pd.concat([sample['id'].astype(int), result],axis=1)
submit.to_csv('data/submission1.csv', index=False)

[False False  True ... False False False]


c:\Users\nakamura\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
# 2回目 
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 特徴量と目的変数の分割
X = train[['l','v(g)','branchCount','loc']]
y = train['defects']

# データの分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.666667, random_state=42, shuffle=True)

# モデルの学習
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X_train, y_train)

# 評価用データの予測
X_test = test[['l','v(g)','branchCount','loc']]
y_pred = model.predict(X_test)

# 予測結果の表示
print(y_pred)

# 結果をCSVファイルとして保存
result = pd.DataFrame(y_pred).astype(int)
result.columns = ['defects']
submit = pd.concat([sample['id'].astype(int), result],axis=1)
submit.to_csv('data/submission2.csv', index=False)

[False False  True ... False False  True]


In [6]:
# 3回目 
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

# 特徴量と目的変数の分割
X = train[['l','v(g)','branchCount','loc']]
y = train['defects']

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

# モデルの学習のためのパラメータ設定
params_xgb = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'subsample': 0.6,
    'min_child_weight': 10,
    'max_depth': 9,
    'lambda': 3,
    'gamma': 0,
    'colsample_bytree': 0.7,
    'alpha': 0.2
}

# XGBoostモデルの学習
num_round = 100
model_xgb = xgb.XGBClassifier(**params_xgb)
model_xgb.fit(X_train, y_train)

# ロジスティック回帰モデルの学習
model_lr = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
model_lr.fit(X_train, y_train)

# スタッキングでアンサンブル
estimators = [('xgb', model_xgb), ('lr', model_lr)]
stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

# スタッキングモデルの学習
stacking_model.fit(X_train, y_train)

# 評価用データの予測
y_pred_proba = stacking_model.predict_proba(X_test)[:, 1]  # クラス1の予測確率を取得
y_pred = (y_pred_proba > 0.5).astype(bool)  # 0.5を閾値にして0または1に変換
print(y_pred_proba)
print(y_pred)

# モデルの評価
accuracy = accuracy_score(y_test, y_pred)
macro_f1 = f1_score(y_test, y_pred, average='macro')
classification_rep = classification_report(y_test, y_pred, digits=3)

# 結果の出力
print('Accuracy(test): %.5f' % accuracy)
print('macro-F1:', macro_f1)
print(classification_rep)

# 評価用データの予測
test = test[['l','v(g)','branchCount','loc']]
y_pred_proba = stacking_model.predict_proba(test)[:, 1]
y_pred = (y_pred_proba > 0.5).astype(bool)  # 0.5を閾値にして0または1に変換

# 予測結果の表示
print(y_pred)

# 結果をCSVファイルとして保存
result = pd.DataFrame(y_pred).astype(int)
result.columns = ['defects']
submit = pd.concat([sample['id'].astype(int), result],axis=1)
submit.to_csv('data/submission3.csv', index=False)

[0.10261566 0.43824746 0.13677403 ... 0.12515194 0.11327329 0.18928498]
[False False False ... False False False]
Accuracy(test): 0.81153
macro-F1: 0.6688337256070771
              precision    recall  f1-score   support

       False      0.831     0.949     0.886     31480
        True      0.663     0.342     0.451      9226

    accuracy                          0.812     40706
   macro avg      0.747     0.646     0.669     40706
weighted avg      0.793     0.812     0.788     40706

[False False  True ... False False  True]
